# DCOPF Verification

Prepared by [Jinning Wang](https://jinningwang.github.io).

## Conclusion

For test cases, DCOPF results from AMS are identical to that from MATPOWER.

In [1]:
import datetime

import numpy as np
import pandas as pd

import ams

In [2]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'ams: {ams.__version__}')

Last run time: 2024-03-02 16:57:25
ams: 0.9.1


Using built-in MATPOWER cases as inputs.

In [3]:
cases = [
    ams.get_case('matpower/case14.m'),
    ams.get_case('matpower/case39.m'),
    ams.get_case('matpower/case118.m'),
    ams.get_case('npcc/npcc.m'),
    ams.get_case('wecc/wecc.m'),
    ams.get_case('matpower/case300.m'),]

case_names = [case.split('/')[-1].split('.')[0] for case in cases]

In [4]:
ams_obj = np.zeros(len(cases))

for i, case in enumerate(cases):
    sp = ams.load(case, setup=True)
    sp.DCOPF.init()
    sp.DCOPF.solve(solver='ECOS')
    ams_obj[i] = sp.DCOPF.obj.v

Following MATPOWER results are obtained using MATPOWER 8.0b1 and Matlab R2023b.

In [5]:
mp_obj = np.array([7642.59177699,  41263.94078588,
                   125947.8814179, 705667.88555058,
                   348228.35589771, 706292.32424361])

In [6]:
res = pd.DataFrame({'AMS': ams_obj, 'MATPOWER': mp_obj},
                   index=case_names)
res

,AMS,MATPOWER
case14,7642.591787,7642.591777
case39,41263.942507,41263.940786
case118,125947.880575,125947.881418
npcc,705667.885563,705667.885551
wecc,348228.355882,348228.355898
case300,706292.325366,706292.324244
